In [18]:
import files2dataframe
reload(files2dataframe)
files = ['test.log', 'test.log']
df = files2dataframe.files_to_df(files)
df

,rs,ndim,kmax,kf,deltaK,dav_its,Virtual States,Vir Energies,Occupied States,Occ Energies,Nvir,Nocc,Nk,Nexc,Kgrid,Excitations,Davidson lowest eigenvalues at each iteration,Dav_Lowest_Val,All Davidson Eigenvalues at Last Iteration
0,1.2,2,2.357023,1.178511,0.471405,8,"[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[-0.0686323805205, -0.233601864085, -0.0686323...",79,21,11,117,"[-2.35702260396, -1.88561808316, -1.4142135623...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [...","[0.321551633699, 0.23730703994, 0.116337773186...",0.111856,"[0.111856109137, 0.173802170042, 0.17380217005..."
1,1.2,2,2.357023,1.178511,0.471405,8,"[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5...","[5.29184839243, 4.28253149811, 3.47787501612, ...","[[3, 4], [3, 5], [3, 6], [4, 3], [4, 4], [4, 5...","[-0.0686323805205, -0.233601864085, -0.0686323...",79,21,11,117,"[-2.35702260396, -1.88561808316, -1.4142135623...","[[0, 63], [0, 73], [0, 4], [0, 14], [0, 24], [...","[0.321551633699, 0.23730703994, 0.116337773186...",0.111856,"[0.111856109137, 0.173802170042, 0.17380217005..."
